In [ ]:
from google.colab import drive 
drive.mount ('/content/drive/')

In [ ]:
PROJECT_PATH= "/content/drive/My Drive//PascalVOC2012_Dataset/"  

In [ ]:
import torch
from torchvision import datasets, transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
# train_transform = transforms.Compose([transforms.Resize((128, 128),transforms.ToTensor())]) 

# test_transform = transforms.Compose ([transforms.Resize((128,128), transforms.ToTensor())])  

# train = datasets.VOCSegmentation(root= 'data', image_set='train', download=True, transform=train_transform, target_transform=train_transform, transforms=None)
# test= datasets.VOCSegmentation(root= 'data', image_set='val', download=True, transform=test_transform, target_transform=test_transform, transforms=None)



In [ ]:
import torch.utils.data as data
from PIL import Image, ImageDraw
import os
import os.path

class VOCSegmentation(data.Dataset):
    def __init__(self, root, image_set, transform=None, target_transform=None):
        self.root = root
        self.image_set = image_set
        self.transform = transform
        self.target_transform = target_transform

        self._annopath = os.path.join(self.root, 'SegmentationClass', '%s.png')
        self._imgpath = os.path.join(self.root, 'JPEGImages', '%s.jpg')
        self._imgsetpath = os.path.join(self.root, 'ImageSets', 'Segmentation', '%s.txt')
 
        with open(self._imgsetpath % self.image_set) as f:
            self.ids = f.readlines()
        self.ids = [x.strip('\n') for x in self.ids]

    def __getitem__(self, index):
        img_id = self.ids[index]

        target = Image.open(self._annopath % img_id)#.convert('RGB')

        img = Image.open(self._imgpath % img_id).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.ids)


In [ ]:
train_transform = transforms.Compose([transforms.Resize((128, 128)),transforms.ToTensor()]) 

test_transform = transforms.Compose ([transforms.Resize((128,128)), transforms.ToTensor()])  

training_set = VOCSegmentation(root= PROJECT_PATH, image_set = 'train', transform= test_transform, target_transform= test_transform)
testing_set = VOCSegmentation(root= PROJECT_PATH, image_set = 'val', transform= test_transform, target_transform= test_transform)

#training_set = datasets.VOCSegmentation(root= 'data', image_set = 'train', download=True, transform= train_transform, target_transform= train_transform)
#testing_set = datasets.VOCSegmentation(root= 'data', image_set = 'val', download=True, transform= test_transform, target_transform= test_transform)

# x,y=training_set[0]
# print(x[0,0:4,0:4])
# print(y[0,0:4,0:4])


In [ ]:

print(f"trainset length: {len(training_set)}")
print(f"valset length: {len(testing_set)}")


In [ ]:
#training_set.ids = training_set.ids
training_set.ids = training_set.ids[:16]
len(training_set)

In [ ]:

batch_size = 16

train_loader = torch.utils.data.DataLoader(training_set, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(testing_set, batch_size, shuffle=True)


dataiter = iter(train_loader)
#get first batch of the dataset
images, labels = dataiter.next()


batch_index = 0

print(images[batch_index].shape)
print(labels[batch_index].shape)
print(images.shape)
#torch.set_printoptions(profile="full")
#torch.set_printoptions(edgeitems=3)
#print(labels[0]*255)  

def show(img):
    plt.imshow(np.transpose(img, (1, 2, 0)), interpolation='nearest')

def show_gray(img):
    plt.imshow(np.transpose(img, (1, 2, 0)), interpolation='nearest', cmap='gray')

label_rgb =  labels[batch_index]
label_rgb = torch.cat([label_rgb,label_rgb,label_rgb], dim=0)


print(label_rgb.shape)


show(images[0])  
  

In [ ]:

def cmap(n):
    if (n is None) or (n == 0):
        n = 255

    cmap_array = np.zeros((n, 3))
    for i in range(n):
        id = i
        r = 0
        g = 0
        b = 0
        for j in range(7):
            bgr_string = np.binary_repr(id)
            bgr = int(bgr_string[-1])
            bsr = np.left_shift(bgr, 7 - j)
            r = np.bitwise_or(r, bsr)

            bgg_string = np.binary_repr(id)
            if len(bgg_string) > 1:
                bgg = int(bgg_string[-2])
            else:
                bgg = 0
            bsg = np.left_shift(bgg, 7 - j)
            g = np.bitwise_or(g, bsg)

            bgb_string = np.binary_repr(id)
            if len(bgb_string) > 2:
                bgb = int(bgb_string[-3])
            else:
                bgb = 0
            bsb = np.left_shift(bgb, 7 - j)
            b = np.bitwise_or(b, bsb)

            id = np.right_shift(id, 3)
        cmap_array[i, 0] = r
        cmap_array[i, 1] = g
        cmap_array[i, 2] = b

    return cmap_array

cmap_array_gen = cmap(21)
print("1")

In [ ]:


def get_rgb_img_from_raw_tensor(input_tensor, batch_index):
  """
  logic to convert to rgb image:
  Args:

    input_tensor:  Tensor [B,H,W] with values between [0-19,255] PASCALVOC2007 format
    batch_index: convert index to rgb from batch (B available images)
  """

  rgb_tensor = np.zeros((128, 128, 3))
  #print(rgb_tensor.shape)
  for i in range(128):
      for j in range(128):
          rgb = cmap_array_gen[input_tensor[batch_index, i, j]]
          rgb_tensor[i, j, :] = np.flipud(rgb)

  #print(rgb_tensor.shape)

  pil_image = Image.fromarray(np.uint8(rgb_tensor))
  #print(pil_image.size)
  return pil_image


In [ ]:
def process_labels(labels):
  """
  recover [0-20 and 255] initial gt format from [0-1] by multiplying with 255
  remove additional dimension at position 1 (channel) using squeeze  : B,1,H,W (dataloader) ->B,H,W (format expected by crossentropy clsindex)
  Args:

    input_tensor:  Tensor [B,H,W] with values between [0-19,255] PASCALVOC2007 format
    batch_index: convert index to rgb from batch (B available images)
  """

  
  #print(f"process_labels before {labels.shape}")
  labels=labels.squeeze(1)*255
  #print(f"process_labels after squeeze {labels.shape}")
  labels = labels.type(torch.LongTensor)
  #void pixels (boundary region) is considered background
  undefined_indices = labels == 255 
  labels[undefined_indices] = 0
  return labels



In [ ]:

labels = process_labels(labels)
pil_target_image = get_rgb_img_from_raw_tensor(labels,batch_index)
pil_target_image

In [ ]:
show(images[batch_index]) 

In [ ]:
show_gray(label_rgb) 

In [ ]:
 
import torch 
import torch.nn as nn

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 24)
        self.down1 = Down(24, 48)
        self.down2 = Down(48, 96)
        self.down3 = Down(96, 192)
        factor = 2 if bilinear else 1
        self.down4 = Down(192, 384 // factor)
        self.up1 = Up(384, 192 // factor, bilinear)
        self.up2 = Up(192, 96 // factor, bilinear)
        self.up3 = Up(96, 48 // factor, bilinear)
        self.up4 = Up(48, 24, bilinear)
        self.outc = OutConv(24, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

    
image = torch.rand((1, 3, 128, 128))
model = UNet(3,21)
out = model(image)
print(out.size())

In [ ]:
from torchsummary import summary
summary(model, input_size=(3, 128, 128))

In [ ]:
from torch import optim
import torch.nn 
import multiprocessing


num_worker=int(multiprocessing.cpu_count()/2)
if torch.cuda.is_available():
   device = torch.device('cuda')
   torch.cuda.empty_cache()
   print("Cuda")

else:
   device = torch.device('cpu')

# model = Unet(nclasses)
# if os.path.exists(Model_Path):
#    model.load_state_dict(torch.load(Model_Path))

# model=model.to(device)
# optimizer = torch.optim.SGD(model.parameters(),lr = Lr, momdentum=momentum)
# criterion = torch.nn.CrossEntropyLoss()

  

In [ ]:
SAVEPATH = '/content/drive/My Drive//ModelSaved/model_3.pth'

def train_model(model, dataloaders, optimizer, scheduler, device, num_epochs=200):
    best_model_wts = copy.deepcopy(model.state_dict())
    avg_best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()
      
        # Each epoch has a training and validation phase
        phases=['train']
        if 'val' in dataloaders.keys():
          phases.append('val')
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for iteration,(inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = process_labels(labels)
                # #recover [0-20 and 255] initial gt format from [0-1] by multiplying with 255
                # #remove additional dimension at position 1 (channel) using squeeze  : B,1,H,W (dataloader) ->B,H,W (crossentropy loss format clsindex)
                # labels = labels.squeeze(1)*255
                # labels = labels.type(torch.LongTensor)
                # #void pixels (boundary region) is considered background
                # labels[labels == 255] = 0
                
                labels = labels.to(device)


                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    print(outputs.shape)
                    print(labels.shape)
                    loss = calc_loss(outputs, labels, metrics)  #labels is clsindex for crossentropy loss [0-19]
                    #if iteration%10==0:
                    print(f"{phase}_loss at epoch :{epoch}/ iteration: {iteration}/{len(dataloaders[phase])} loss: {loss}")
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                if phase == 'train':
                    scheduler.step()
                    for param_group in optimizer.param_groups:
                        print("LR", param_group['lr'])
                # statistics
                epoch_samples += inputs.size(0)

            print(f"{phase}_loss at end of epoch :{epoch}  loss: {loss}")
            #print_metrics(metrics, epoch_samples, phase)
            #AVERAGE LOSS Metric (for validation) PER EPOCH
            avg_epoch_loss = metrics['loss'] / epoch_samples



            # deep copy the model
            if phase == 'val' and avg_epoch_loss < avg_best_loss:
                print("saving best model")
                avg_best_loss = avg_epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                #torch.save(model.state_dict(), SAVEPATH)

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        if epoch % 5==0:
          torch.save(model.state_dict(), SAVEPATH)
          print("saving model pth")
    print('Best val loss: {:4f}'.format(avg_best_loss))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import collections
d = collections.defaultdict(list)

# def dice_loss(pred, target, smooth = 1.):
#     pred = pred.contiguous()
#     target = target.contiguous()    

#     intersection = (pred * target).sum(dim=2).sum(dim=2)
    
#     loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
#     return loss.mean()
criterion = nn.CrossEntropyLoss()
 

def calc_loss(pred, target, metrics, bce_weight=0.5):
    
    CCE_loss = criterion(pred, target)
    metrics['CCE'] += CCE_loss.data.cpu().numpy() * target.size(0)

    return CCE_loss

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        

    print("{}: {}".format(phase, ", ".join(outputs)))

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from collections import defaultdict


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_classes = 21
#model = Unet1(num_classes).to(device)
model = UNet(3,num_classes).to(device)
#model = U_Net3(3,num_classes).to(device)

optimizer_ft = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-2)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.1)

#dataloaders={'train':train_loader, 'val':val_loader}

#for overfitting on truncated dataset do not validate after each epoch
dataloaders={'train':train_loader}
print(dataloaders)
model = train_model(model, dataloaders, optimizer_ft, exp_lr_scheduler, device=device, num_epochs=200)

In [ ]:
dataloaders={'train':train_loader}
'val' in dataloaders.keys()

In [ ]:
testiter = iter(val_loader)



In [ ]:
trainiter = iter(train_loader)
print(len(train_loader.dataset))

In [ ]:
#get first batch of the dataset
#images, labels = testiter.next()
images, labels = trainiter.next()

In [ ]:
# print input
val_images=images
print(val_images.shape)
show(val_images[0])


In [ ]:
#print target 
print(labels.shape)
labels = process_labels(labels)
print(labels.shape)
rgb_image=get_rgb_img_from_raw_tensor(labels,0)
rgb_image

In [ ]:
print(labels[0,100,100])
print(labels[0,120,35])

In [ ]:
#laoding model from Drive
SAVEPATH = '/content/drive/My Drive//ModelSaved/model_3.pth'
import os.path
from os import path
print(path.exists(SAVEPATH))
model = UNet(3,21)

model.load_state_dict(torch.load(SAVEPATH))
model.eval()

In [ ]:
#add batch dimension
#val_image=val_image.unsqueeze(0)

#torch load(mo...dsdsta path)
print(val_images.shape)
out=model(val_images) #B,21,H,W ->B,1,H,W->B,H,W   [0,0,0,0,1,00,0,...0] -> 5
#softmaxlayer = nn.Softmax(dim=1)
#out=softmaxlayer(out) #maybe not necessary- because does not change argmax result 
print(out.shape)
out_argmax= torch.argmax(out,1)
print(out_argmax.shape)


In [ ]:
#print output
rgb_image=get_rgb_img_from_raw_tensor(out_argmax,0)
rgb_image

In [ ]:
torch.max(out)
print(out[0,:,100,100])
print(out_argmax[0,100,100])

print(out[0,:,20,60])
print(out_argmax[0,20,60])